# VacationPy
----

The below scripts explores how to take data from a dataframe, that was imported by a csv, and create an interactive map using the Google Maps API. A heatmap was generated using the location of 579 cities in our dataframe and showcasing the humidity. Another heatmap was generated with markers of 11 cities of ideal weather conditions and the name of a hotel in a 5000 mile radius of the city. I used Matplotlib, Pandas, Numpy, Request, OS, Google Maps API to accompolish our task.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../output_data/cities.csv")
weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hambantota,6.1241,81.1185,73.74,100,20,2.30,LK,1667181777
1,Hilo,19.7297,-155.0900,81.25,93,36,3.00,US,1667181777
2,Fortuna,40.5982,-124.1573,54.09,85,19,1.99,US,1667181778
3,Qaanaaq,77.4840,-69.3632,-6.11,71,7,7.96,GL,1667181778
4,Mar del Plata,-38.0023,-57.5575,50.88,71,6,8.99,AR,1667181779
...,...,...,...,...,...,...,...,...,...
574,Ystad,55.4297,13.8204,54.37,91,60,3.56,SE,1667182087
575,Itamarandiba,-17.8572,-42.8589,66.33,96,98,1.74,BR,1667182087
576,Reconquista,-29.1500,-59.6500,56.50,66,100,19.69,AR,1667182088
577,Kaoma,-14.7833,24.8000,65.68,69,100,3.40,ZM,1667182088


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store the latitude and longitude in locations and store the humidity to be used as the weight
locations = weather_data[["Lat", "Lng"]].astype(float)
humidity = weather_data["Humidity"].astype(float)

In [4]:
# Plot Heatmap 
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, 
                                 max_intensity = max(humidity), point_radius = 3)

# Add heat layer 
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
# Narrow down cities for ideal weather conditions and drop any null values
ideal_cities = weather_data.loc[(weather_data["Max Temp"] <= 82) & (weather_data["Max Temp"] >= 70) 
                                & (weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0)].dropna()

# Reset the index
ideal_cities = ideal_cities.reset_index(drop = True)

# Display results
ideal_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Saint-Pierre,-21.3393,55.4781,72.91,73,0,0.00,RE,1667181783
1,Sur,22.5667,59.5289,74.70,65,0,6.91,OM,1667181833
2,Khipro,25.8285,69.3772,77.18,36,0,5.26,PK,1667181839
3,Tura,25.5198,90.2201,73.60,78,0,2.04,IN,1667181878
4,Kodīnar,20.7903,70.7031,75.92,45,0,7.99,IN,1667181894
5,Beloha,-25.1667,45.0500,72.32,63,0,7.94,MG,1667181902
6,Quelimane,-17.8786,36.8883,73.56,90,0,4.18,MZ,1667181980
7,Sola,-13.8833,167.5500,81.36,68,0,1.90,VU,1667181997
8,Arona,28.0996,-16.6810,70.92,66,0,5.75,ES,1667182001
9,Sotouboua,8.5667,0.9833,72.77,59,0,0.76,TG,1667182002


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
# Store into a variable named hotel_df
hotel_df = ideal_cities

# Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""

# Display the results
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Saint-Pierre,-21.3393,55.4781,72.91,73,0,0.00,RE,1667181783,
1,Sur,22.5667,59.5289,74.70,65,0,6.91,OM,1667181833,
2,Khipro,25.8285,69.3772,77.18,36,0,5.26,PK,1667181839,
3,Tura,25.5198,90.2201,73.60,78,0,2.04,IN,1667181878,
4,Kodīnar,20.7903,70.7031,75.92,45,0,7.99,IN,1667181894,
5,Beloha,-25.1667,45.0500,72.32,63,0,7.94,MG,1667181902,
6,Quelimane,-17.8786,36.8883,73.56,90,0,4.18,MZ,1667181980,
7,Sola,-13.8833,167.5500,81.36,68,0,1.90,VU,1667181997,
8,Arona,28.0996,-16.6810,70.92,66,0,5.75,ES,1667182001,
9,Sotouboua,8.5667,0.9833,72.77,59,0,0.76,TG,1667182002,


In [17]:
# Set base url and params
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key }

# Use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    # Get the latitude and longitude from the dataframe
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Assemble url and make API request
    response = requests.get(base_url, params = params).json()
    
    # Extract results
    results = response["results"]
    
    # Place results in Hotel Name column while handling errors
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
        
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"]= "Hotel not available"
      
        
# Display the results
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Saint-Pierre,-21.3393,55.4781,72.91,73,0,0.00,RE,1667181783,Villa Delisle
1,Sur,22.5667,59.5289,74.70,65,0,6.91,OM,1667181833,Sur Grand Hotel
2,Khipro,25.8285,69.3772,77.18,36,0,5.26,PK,1667181839,New Mehran Hotel & Marvi Musafir Khana
3,Tura,25.5198,90.2201,73.60,78,0,2.04,IN,1667181878,Hotel Polo Orchid
4,Kodīnar,20.7903,70.7031,75.92,45,0,7.99,IN,1667181894,Hotel Sun Shine Kodinar
5,Beloha,-25.1667,45.0500,72.32,63,0,7.94,MG,1667181902,Hotel not available
6,Quelimane,-17.8786,36.8883,73.56,90,0,4.18,MZ,1667181980,Hotel Elite
7,Sola,-13.8833,167.5500,81.36,68,0,1.90,VU,1667181997,Hotel not available
8,Arona,28.0996,-16.6810,70.92,66,0,5.75,ES,1667182001,Iberostar Selection Sábila
9,Sotouboua,8.5667,0.9833,72.77,59,0,0.76,TG,1667182002,Hotel Sarakawa


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))